In [44]:
import pandas as pd


sightings = pd.read_csv('koala-survey-sightings-data.csv', encoding='utf-8', parse_dates=[['Date', 'Time']])

# They used a shorthand for some of the genus
sightings['TreeSpecies'] = sightings['TreeSpecies'].replace({"E.": "Eucalyptus", "L.":"Lophostemon", "C.": "Corymbia", "A.": "Angophora"}, regex=True)
sightings['TreeSpecies'] = sightings['TreeSpecies'].dropna()
sightings.drop_duplicates(subset=['TreeSpecies'], inplace=True)

sightings[['TreeSpecies']].to_csv('koala-sightings-species.csv')
# ^ Upload that file to the ALA and download the results and field guide with a mapping to the correct species.
#sightings_with_species_info = pd.merge(sightings, unique_df, right_on="Scientific Name - original", left_on="TreeSpecies", how="left")

In [45]:
import requests
from urllib.parse import urlsplit
import os
processed_df = pd.read_csv('SEQ_Koala_Survey_Data_2010_-_2015.csv')

for _, row in processed_df.iterrows():
    doc_id = urlsplit(row["guid"]).path.split('/')[-1]
    r = requests.get(row["guid"] + ".json")
    with open(os.path.join('apni', doc_id + '.json'), 'wb') as f:
        f.write( r.content )

In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlsplit
import os

processed_df = pd.read_csv('SEQ_Koala_Survey_Data_2010_-_2015.csv')

for _, row in processed_df.iterrows():
    try:
        doc_id = urlsplit(row["guid"]).path.split('/')[-1]
        path = os.path.join('apni', doc_id + '.txt')
        if not os.path.exists(path):
            response = requests.get(
                url="https://en.wikipedia.org/wiki/{0}".format(row["scientificName"].replace(" ", "_")),
            )
            soup = BeautifulSoup(response.content, 'html.parser')
            d = soup.find(id="Description").parent.next_sibling.next_sibling.get_text()
            with open(os.path.join('apni', doc_id + '.txt'), 'w') as f:
                f.write(d)
        else:
            print(path, " already exists")
    except Exception as e:
        print(e, row[0])

apni/2916005.txt  already exists
apni/2895329.txt  already exists
apni/2899524.txt  already exists
apni/2887514.txt  already exists
apni/51290068.txt  already exists
apni/2914381.txt  already exists
apni/2914136.txt  already exists
apni/2897003.txt  already exists
apni/2892322.txt  already exists
apni/51258114.txt  already exists
apni/2895194.txt  already exists
'NoneType' object has no attribute 'parent' Lophostemon suaveolens
apni/2906254.txt  already exists
apni/2908951.txt  already exists
apni/2897003.txt  already exists
apni/2915125.txt  already exists
apni/2887103.txt  already exists
apni/51289954.txt  already exists
apni/2912509.txt  already exists
apni/2919306.txt  already exists
apni/51302291.txt  already exists
apni/2903018.txt  already exists
apni/2904406.txt  already exists
apni/7622844.txt  already exists
apni/2914034.txt  already exists
apni/2914283.txt  already exists
apni/2918082.txt  already exists
'NoneType' object has no attribute 'parent' Corymbia Callistemon
apni/2

In [16]:
# generate plants.json
processed_df = pd.read_csv('SEQ_Koala_Survey_Data_2010_-_2015.csv')
index = {}
for _, row in processed_df.iterrows():
    doc_id = urlsplit(row["guid"]).path.split('/')[-1]
    path = os.path.join('apni', doc_id + '.txt')
    index[row['Supplied Name']] = {'id': doc_id, 'scientificName': row['scientificName'], 'family': row['family'] }
    print('<option value="{0}">{1}</option>'.format(row['Supplied Name'], row['Supplied Name']))
    
import json
with open('plants.json', 'w') as f:
    f.write(json.dumps(index))


'{"Eucalyptus carnea": {"id": "2916005", "scientificName": "Eucalyptus carnea", "family": "Myrtaceae"}, "Eucalyptus microcorys": {"id": "2895329", "scientificName": "Eucalyptus microcorys", "family": "Myrtaceae"}, "Alphitonia excelsa": {"id": "2899524", "scientificName": "Alphitonia excelsa", "family": "Rhamnaceae"}, "Melaleuca quinquenervia": {"id": "2887514", "scientificName": "Melaleuca quinquenervia", "family": "Myrtaceae"}, "Eucalyptus tereticornis": {"id": "51290068", "scientificName": "Eucalyptus tereticornis", "family": "Myrtaceae"}, "Eucalyptus siderophloia": {"id": "2914381", "scientificName": "Eucalyptus siderophloia", "family": "Myrtaceae"}, "Eucalyptus racemosa": {"id": "2914136", "scientificName": "Eucalyptus racemosa", "family": "Myrtaceae"}, "Corymbia trachyphloia": {"id": "2897003", "scientificName": "Corymbia trachyphloia", "family": "Myrtaceae"}, "Eucalyptus littoralis": {"id": "2891244", "scientificName": "Eucalyptus litoralis", "family": "Myrtaceae"}, "Eucalyptus s